# LAMMPS Nanotube Visualization
Setup: Install dependencies, load data, configure parameters

In [ ]:
import sys, subprocess, os
def install(pkg): 
    try: __import__(pkg.split('>=')[0])
    except: subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", pkg])
[install(p) for p in ["MDAnalysis>=2.4.0", "matplotlib>=3.5.0", "imageio>=2.15.0", "pandas>=1.3.0"]]

import MDAnalysis as mda, matplotlib.pyplot as plt, imageio, numpy as np, pandas as pd, matplotlib.gridspec as gridspec
from MDAnalysis.lib.distances import distance_array

LEFT_FORCE_DURATION, LEFT_VELOCITY = 20000, 0.003
u = mda.Universe("trajectory.lammpstrj", format="LAMMPSDUMP")
temp_df = pd.read_csv('temperature.csv')
steps, temps = temp_df['Step'].values, temp_df['Temperature'].values
total_sim_time = int(steps.max())
force_zero_timestep = 100 + LEFT_FORCE_DURATION + 600

distances = distance_array(u.atoms.positions, u.atoms.positions)
bond_pairs = [(i, j) for i in range(len(u.atoms)) for j in range(i+1, len(u.atoms)) if 1.2 <= distances[i, j] <= 1.8]
if bond_pairs: u.add_TopologyAttr('bonds', np.array(bond_pairs))

all_x = np.concatenate([ts.positions[:, 0] for ts in u.trajectory])
all_y = np.concatenate([ts.positions[:, 1] for ts in u.trajectory])
xlim, ylim = (all_x.min(), all_x.max()), (all_y.min(), all_y.max())
u.trajectory.rewind()

Generate: Create animation frames with bonds, arrows, and temperature plot → cnt_trajectory.gif

In [ ]:
images, N, arrow_length, decay = [], 10, 30, 5

for i, ts in enumerate(u.trajectory):
    if i % 20 != 0: continue
    positions, cnt_top, cnt_bot = u.atoms.positions, u.atoms[:N], u.atoms[-N:]
    fig = plt.figure(figsize=(8, 6))
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1], hspace=0.3)
    ax_main, ax_life = fig.add_subplot(gs[0]), fig.add_subplot(gs[1])
    
    mean_y = np.mean(positions[:, 1])
    for bond in u.bonds:
        pos1, pos2 = bond.atoms.positions
        alpha = np.exp(-np.abs(np.mean([pos1[1], pos2[1]]) - mean_y)/decay)
        ax_main.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], color='hotpink', linewidth=2, alpha=alpha)
    
    center_y, left_x, right_x = np.mean(positions[:, 1]), np.mean(cnt_top.positions[:, 0]), np.mean(cnt_bot.positions[:, 0])
    current_step = int(ts.time)
    if current_step < force_zero_timestep:
        ax_main.arrow(left_x, center_y, -arrow_length/3, 0, width=1, color='blue', head_width=4, head_length=4)
    ax_main.arrow(right_x, center_y, arrow_length/3, 0, width=1, color='blue', head_width=4, head_length=4)
    
    ax_main.set_title(f"Timestep {ts.time:.0f}", fontsize=18)
    ax_main.axis('off')
    ax_main.set_xlim(xlim[0] - 50, xlim[1] + 50)
    ax_main.set_ylim(ylim)
    
    idx = (steps <= current_step)
    ax_life.set_xlim(0, total_sim_time)
    ax_life.plot(steps, temps, color='gray', alpha=0.3)
    ax_life.plot(steps[idx], temps[idx], color='red', linewidth=2)
    ax_life.set_ylabel('Temp')
    ax_life.set_xlabel('Step')
    ax_life.set_ylim(temps.min(), temps.max())
    ax_life.grid(True, alpha=0.2)
    
    plt.tight_layout(pad=0.5)
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.buffer_rgba(), dtype='uint8').reshape(fig.canvas.get_width_height()[::-1] + (4,))
    images.append(image[..., :3])
    plt.close(fig)

if images:
    imageio.mimsave('cnt_trajectory.gif', images, duration=1/300, loop=0)
    print(f"Generated cnt_trajectory.gif with {len(images)} frames")
else:
    print("No images generated")